In [ ]:
%matplotlib inline
!pip install -U git+https://github.com/albu/albumentations --no-cache-di
!pip install segmentation-models-pytorch
!pip install psutil
!pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12==24.8.* dask-cudf-cu12==24.8.* cuml-cu12==24.8.* cugraph-cu12==24.8.* cuspatial-cu12==24.8.* cuproj-cu12==24.8.* cuxfilter-cu12==24.8.* cucim-cu12==24.8.* pylibraft-cu12==24.8.* raft-dask-cu12==24.8.* cuvs-cu12==24.8.*


In [ ]:
import os
import sys
import cv2
import gdown
import numpy as np
import pandas as pd
from glob import glob
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler
import json
import logging
import time
import psutil
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import segmentation_models_pytorch as smp
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
# Define a new dataset to only return images (without masks)
class ImageDataset:
    def __init__(self, dataframe, transform=None):
        self.dataset = dataframe
        self.transform = transform

    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, index):
        df_row = self.dataset.iloc[index]
        vv_image = cv2.imread(df_row['vv_image_path'], 0) / 255.0
        vh_image = cv2.imread(df_row['vh_image_path'], 0) / 255.0
        rgb_image = s1_to_rgb(vv_image, vh_image)
        
        if self.transform:
            augmented = self.transform(image=rgb_image)
            rgb_image = augmented['image']

        return {'image': rgb_image.transpose((2,0,1)).astype('float32')}


class ETCIDataset(Dataset):
    def __init__(self, dataframe, split, transform=None):
        self.split = split
        self.dataset = dataframe
        self.transform = transform

    def __len__(self):
        return self.dataset.shape[0]


    def __getitem__(self, index):
        example = {}

        df_row = self.dataset.iloc[index]

        # load vv and vh images
        vv_image = cv2.imread(df_row['vv_image_path'], 0) / 255.0
        vh_image = cv2.imread(df_row['vh_image_path'], 0) / 255.0

        # convert vv and ch images to rgb
        rgb_image = s1_to_rgb(vv_image, vh_image)

        if self.split == 'test':
            # no flood mask should be available
            example['image'] = rgb_image.transpose((2,0,1)).astype('float32')
        else:
            # load ground truth flood mask
            flood_mask = cv2.imread(df_row['flood_label_path'], 0) / 255.0

            # compute transformations
            if self.transform:
                augmented = self.transform(image=rgb_image, mask=flood_mask)
                rgb_image = augmented['image']
                flood_mask = augmented['mask']

            example['image'] = rgb_image.transpose((2,0,1)).astype('float32')
            example['mask'] = flood_mask.astype('int64')

        return example

def prepare_dataset_paths(dset_root):
    train_dir = os.path.join(dset_root, 'train')
    n_train_regions = len(glob(train_dir + '/*/'))

    vv_image_paths = sorted(glob(train_dir + '/**/vv/*.png', recursive=True))
    vv_image_names = [get_filename(pth) for pth in vv_image_paths]
    region_name_dates = ['_'.join(n.split('_')[:2]) for n in vv_image_names]

    vh_image_paths, flood_label_paths, region_names = [], [], []
    for i in range(len(vv_image_paths)):
        vh_image_name = vv_image_names[i].replace('vv', 'vh')
        vh_image_path = os.path.join(train_dir, region_name_dates[i], 'tiles', 'vh', vh_image_name)
        vh_image_paths.append(vh_image_path)

        flood_image_name = vv_image_names[i].replace('_vv', '')
        flood_label_path = os.path.join(train_dir, region_name_dates[i], 'tiles', 'flood_label', flood_image_name)
        flood_label_paths.append(flood_label_path)

        region_name = region_name_dates[i].split('_')[0]
        region_names.append(region_name)

    return pd.DataFrame({
        'vv_image_path': vv_image_paths,
        'vh_image_path': vh_image_paths,
        'flood_label_path': flood_label_paths,
        'region': region_names
    })


def get_data_subset(train_df, coreset_path=None):
    sub_train_df, _ = train_test_split(train_df, test_size=0.2, random_state=42)
    sub_train_df = sub_train_df.reset_index(drop=False)
    
    if coreset_path is not None:
        with open(coreset_path, 'r') as json_file:
            coreset_indices = json.load(json_file)
        print(f"coreset size: {len(coreset_indices)}")
        sub_train_df = sub_train_df[sub_train_df['index'].isin(coreset_indices)]
   
    return sub_train_df



def get_dataloader(sub_train_df, batch_size, transform=None, coreset_indices=None, strategy="kmeans"):
    
    train_df = sub_train_df
    
    if strategy == "kmeans":
        train_dataset = ImageDataset(train_df)
    else:
        train_dataset = ETCIDataset(train_df, split='train', transform=None)

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=2,
        pin_memory=True
    )

    return train_loader



def set_seed(seed=42):
    generator = torch.Generator().manual_seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    return generator


def get_filename(filepath):
    return os.path.split(filepath)[1]

def s1_to_rgb(vv_image, vh_image):
    ratio_image = np.clip(np.nan_to_num(vh_image/vv_image, 0), 0, 1)
    rgb_image = np.stack((vv_image, vh_image, 1-ratio_image), axis=2)
    return rgb_image


def create_model(device='cuda'):
    # load model from package
    model = smp.Unet(
    encoder_name="resnet34",        
    encoder_weights=None,           
    in_channels=3,                  
    classes=2,                      
    )
    model.load_state_dict(torch.load('model_test_lr_5epochs1.pt')) # load pretrained model

    model.to(device)
    return model


# KMeans Sampling

In [ ]:
import numpy as np
import time
import json
import torch
from tqdm import tqdm
from cuml.cluster import KMeans
from cuml.decomposition import PCA
from torch.utils.data import DataLoader
import numpy as np
import torch
from cuml.decomposition import PCA
import numpy as np



def extract_features(model, dataloader, device):
    model.eval()
    features = []
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(dataloader, desc="Extracting Features")):
            images = batch['image'].to(device)

            # Extract encoder features (last encoder block)
            encoder_features = model.encoder(images)[-1]  

            features.append(encoder_features.cpu().numpy())

    concatenated_features = np.concatenate(features, axis=0)  # Shape (total_samples, channels, H, W)
    
    return concatenated_features

def apply_pca(features, n_components=256):
    num_samples, channels, height, width = features.shape  
    
    reshaped_features = features.transpose(0, 2, 3, 1).reshape(-1, channels) 
        
    pca = PCA(n_components=n_components)
    features_pca = pca.fit_transform(reshaped_features) )
    
    features_pca = features_pca.reshape(num_samples, height, width, n_components)
    
    features_pca = features_pca.reshape(num_samples, -1)
    
    return features_pca



def create_clusters(n_clusters, features):
    print("\nStarting clustering process... ")
    print(f"Feature shape: {features.shape}")
    print(f"Expected number of clusters: {n_clusters}")

    features = np.nan_to_num(features).astype(np.float32)

    start_time = time.time()

    kmeans_cuml = KMeans(n_clusters=n_clusters, random_state=3)
    kmeans_cuml.fit(features)

    cluster_centers = kmeans_cuml.cluster_centers_
    cluster_labels = kmeans_cuml.labels_

    end_time = time.time() - start_time
    print(f"Time taken for clustering: {end_time:.2f} seconds")

    return cluster_centers, cluster_labels


def find_cluster_centers_balanced(features, cluster_centers, cluster_labels, n_clusters):
    coreset_indices = []
    cluster_samples = {i: [] for i in range(n_clusters)}
    
    # Group samples by cluster
    for idx, label in enumerate(cluster_labels):
        cluster_samples[label].append(idx)

    # Identify non-empty clusters
    non_empty_clusters = {k: v for k, v in cluster_samples.items() if len(v) > 0}
    
    # Select one point per non-empty cluster (closest to center)
    for cluster_label, cluster_indices in non_empty_clusters.items():
        center = cluster_centers[cluster_label]
        distances = np.linalg.norm(features[cluster_indices] - center, axis=1)
        closest_index = cluster_indices[np.argmin(distances)]
        coreset_indices.append(closest_index)
    
    # Fill in extra selections using size-weighted sampling from non-empty clusters
    remaining = n_clusters - len(coreset_indices)
    if remaining > 0:
        # Compute selection weights based on cluster sizes
        cluster_ids = list(non_empty_clusters.keys())
        sizes = np.array([len(non_empty_clusters[k]) for k in cluster_ids])
        probabilities = sizes / sizes.sum()
        
        attempts = 0
        while len(coreset_indices) < n_clusters and attempts < 1000:
            selected_cluster = np.random.choice(cluster_ids, p=probabilities)
            candidates = list(set(non_empty_clusters[selected_cluster]) - set(coreset_indices))
            if candidates:
                center = cluster_centers[selected_cluster]
                distances = np.linalg.norm(features[candidates] - center, axis=1)
                closest_index = candidates[np.argmin(distances)]
                coreset_indices.append(closest_index)
            attempts += 1
   
    return coreset_indices



def save_kmeans_coreset(cluster_indices_lists, sub_train_df, filename='samples_kmeans'):
    cluster_indices_lists = sub_train_df["index"].iloc[cluster_indices_lists].tolist()

    # Save the final refined core set indices
    with open(filename+'.json', 'w') as json_file:
        json.dump(cluster_indices_lists, json_file)


def run_kmeans_sampling(subset_input_path=None, coreset_size=1, coreset_output_path="samples_kmeans1"):
    dset_root = 'dataset/ETCI_2021_Competition_Dataset/'
    train_df = prepare_dataset_paths(dset_root)
    full_train_data_len = round(len(train_df)*0.8)

    sub_train_df = get_data_subset(train_df, coreset_path=subset_input_path)


    image_loader = get_dataloader(
        sub_train_df,
        batch_size=64,
        transform=None,
    )

    device = 'cuda'
    model = create_model(device)
    features = extract_features(model, image_loader, device)

    cluster_num = round(full_train_data_len*coreset_size)
    print(f" Number of clusters: {cluster_num}")

    features = np.nan_to_num(features).astype(np.float32)

    features_pca = apply_pca(features, n_components=128)

    # Run clustering
    cluster_centers, cluster_labels = create_clusters(cluster_num, features_pca)

    cluster_indices_lists = find_cluster_centers_balanced(features_pca, cluster_centers, cluster_labels, cluster_num)

    save_kmeans_coreset(cluster_indices_lists, sub_train_df, coreset_output_path)

    print(f"Core set indices saved to {coreset_output_path}.")
    print("Processing completed successfully.")


In [ ]:
run_kmeans_sampling(subset_input_path="samples/samples_encoder_full_weights_2.json", coreset_size=0.1)

# Gradient Embedding Sampling 

In [ ]:

device = 'cuda'
model = create_model(device)
criterion = nn.CrossEntropyLoss()


### Gradient Embeddings from last Encoder Layer

In [ ]:


def get_grad_embedding_last_encoder_layer(train_dataset, model, criterion, device='cuda'):
    torch.cuda.empty_cache()
    logger = logging.getLogger('GradientEmbeddingLogger')
    logger.setLevel(logging.INFO)

    grad_norms = []  

    model.train()

    loader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=4, pin_memory=True)  # Batch size 1


    # Manually specify the last encoder layer's parameters
    last_encoder_layer = model.encoder.layer4[2].conv2  
    last_encoder_params = list(last_encoder_layer.parameters())[0]  


    for sample_idx, batch in enumerate(tqdm(loader, desc="Computing Gradient Embeddings", leave=True)):
        image = batch['image'].to(device)  # Single sample
        mask = batch['mask'].to(device)  # Ground truth segmentation mask

        model.zero_grad(set_to_none=True)

        # Forward pass
        logits = model(image)

        loss = criterion(logits, mask)  

        loss_scalar = loss.mean()

        last_encoder_params.requires_grad_(True)

        # Compute gradient w.r.t. the last encoder layer
        grad_embedding = torch.autograd.grad(loss_scalar, last_encoder_params, retain_graph=False, create_graph=False)[0]

        # Compute L2 norm of the gradient embedding
        grad_norm = torch.norm(grad_embedding, p=2).item()
        grad_norms.append(grad_norm)


    return grad_norms 


def select_most_influential_samples_encoder(grad_norms, sub_train_df, coreset_size=1, sample_path="coreset_encoder"):
    sub_train_df = sub_train_df.copy() 
    sub_train_df["gradient_norm"] = grad_norms

    # Sort the dataframe in descending order based on gradient_norm
    sorted_df = sub_train_df.sort_values(by="gradient_norm", ascending=False)
    sorted_indices = sorted_df["index"].tolist()
    sorted_indices = sorted_indices[:coreset_size]
    with open(sample_path+'.json', 'w') as json_file:
       json.dump(sorted_indices, json_file)
    
def run_encoder_ge_sampling(subset_input_path=None, coreset_size=1, coreset_output_path="coreset_encoder"):
    dset_root = 'dataset/ETCI_2021_Competition_Dataset/'
    train_df = prepare_dataset_paths(dset_root)
    full_train_data_len = round(len(train_df)*0.8)

    sub_df = get_data_subset(train_df, coreset_path=subset_input_path)
    print(len(sub_df))
    sub_dataset = ETCIDataset(sub_df, split='train', transform=None)
    coreset_size = round(full_train_data_len*coreset_size)
    grad_norms = get_grad_embedding_last_encoder_layer(sub_dataset, model, criterion, 'cuda')
    select_most_influential_samples_encoder(grad_norms, sub_df, coreset_size=coreset_size, sample_path=coreset_output_path)
    print("coreset selection finished")


In [ ]:
run_encoder_ge_sampling(subset_input_path="samples/samples_kmeans_03.json", coreset_size=0.1, coreset_output_path="coreset_encoder1")

### Gradient Embeddings from last Decoder Layer

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import logging

def get_grad_embedding_last_decoder_layer(train_dataset, model, criterion, device='cuda'):
    torch.cuda.empty_cache()
    logger = logging.getLogger('GradientEmbeddingLogger')
    logger.setLevel(logging.INFO)

    grad_norms = []  
    model.train()

    loader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=4, pin_memory=True)  # Batch size 1

    logger.info(f'Starting gradient embedding computation for {len(train_dataset)} samples.')

    # Manually specify the last decoder layer's parameters
    last_decoder_layer = model.decoder.blocks[4].conv2[0]  
    last_decoder_params = list(last_decoder_layer.parameters())[0]  

    logger.info(f"Using last decoder layer: decoder.blocks[4].conv2[0]")

    for sample_idx, batch in enumerate(tqdm(loader, desc="Computing Gradient Embeddings", leave=True)):
        image = batch['image'].to(device)  # Single sample
        mask = batch['mask'].to(device)  # Ground truth segmentation mask

        model.zero_grad(set_to_none=True)

        # Forward pass
        logits = model(image)

        loss = criterion(logits, mask)  

        loss_scalar = loss.mean()

        last_decoder_params.requires_grad_(True)

        # Compute gradient w.r.t. the last decoder layer
        grad_embedding = torch.autograd.grad(loss_scalar, last_decoder_params, retain_graph=False, create_graph=False)[0]

        if grad_embedding is None:
            logger.error(f"No gradients computed for sample {sample_idx}. Check requires_grad settings.")
            continue

        # Compute L2 norm of the gradient embedding
        grad_norm = torch.norm(grad_embedding, p=2).item()
        grad_norms.append(grad_norm)

    logger.info('Completed gradient embedding computation.')

    return grad_norms  # List of L2 norms, one per sample

def select_most_influential_samples_decoder(grad_norms, sub_train_df, coreset_size=1, sample_path="coreset_encoder"):
    sub_train_df = sub_train_df.copy() 
    sub_train_df["gradient_norm"] = grad_norms

    # Sort the dataframe in descending order based on gradient_norm
    sorted_df = sub_train_df.sort_values(by="gradient_norm", ascending=False)
    sorted_indices = sorted_df["index"].tolist()
    sorted_indices = sorted_indices[:coreset_size]
    with open(sample_path+'.json', 'w') as json_file:
       json.dump(sorted_indices, json_file)
    
def run_decoder_ge_sampling(subset_input_path=None, coreset_size=1, coreset_output_path="coreset_encoder"):
    dset_root = 'dataset/ETCI_2021_Competition_Dataset/'
    train_df = prepare_dataset_paths(dset_root)
    full_train_data_len = round(len(train_df)*0.8)

    sub_df = get_data_subset(train_df, coreset_path=subset_input_path)
    print(len(sub_df))
    sub_dataset = ETCIDataset(sub_df, split='train', transform=None)
    coreset_size = round(full_train_data_len*coreset_size)
    grad_norms = get_grad_embedding_last_decoder_layer(sub_dataset, model, criterion, 'cuda')
    select_most_influential_samples_decoder(grad_norms, sub_df, coreset_size=coreset_size, sample_path=coreset_output_path)


In [ ]:
run_decoder_ge_sampling(coreset_size=1, coreset_output_path="coreset_decoder")


Note: For the Hybrid Sampling methods, simply run either KMeans or Gradient Samppling first to save the subset for the first stage. Then load the indices of the subset for the other Sampling method in the second stage to use as the starting dataset.